---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [2]:
# date patterns
# 04/20/2009; 04/20/09; 4/20/09; 4/3/09
# Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;
# 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
# Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
# Feb 2009; Sep 2009; Oct 2010
# 6/2008; 12/2009
# 2009; 2010
# y1983
# r1978
# 4-13-82

In [81]:
def date_sorter():
    
    # env setup
    import pandas as pd
    import numpy as np
    pd.set_option('display.max_rows', 500)
    pd.set_option('display.max_columns', 500)
    pd.set_option('display.width', 1000)

    # load data
    doc = []
    with open('dates.txt') as file:
        for line in file:
            doc.append(line)

    df = pd.Series(doc)

    # cleanup the dates
    # months as string
    result1 = df.str.extractall(r'((?:\d{,2}\s)?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*(?:-|\.|\s|,)\s?\d{,2}[a-z]*(?:-|,|\s)?\s?\d{2,4})')
    # months as numbers
    result2 = df.str.extractall(r'((?:\d{1,2})(?:(?:\/|-)\d{1,2})(?:(?:\/|-)\d{2,4}))')
    # no days (only months & years)
    result3 = df.str.extractall(r'((?:\d{1,2}(?:-|\/))?\d{4})')

    # adjust for typo mistakes
    result1[0] = (result1[0]
                 .str.strip()
                 .str.replace('Decemeber', 'December')
                 .str.replace('Janaury', 'January')
                )

    # add first days or months if info is missing
    result1[0] = pd.to_datetime(result1[0], errors = 'coerce', dayfirst=True)
    result2[0] = pd.to_datetime(result2[0], errors = 'coerce')
    result3[0] = pd.to_datetime(result3[0], errors = 'coerce')

    # merge data 
    result12 = pd.merge(result1, result2, how = 'outer', left_index = True, right_index = True)
    result123 = pd.merge(result12, result3, how = 'outer', left_index = True, right_index = True).reset_index()

    # leave only first occurence of date per one row/sentence/record
    result = result123[result123['match'] == 0]

    # coalesce date columns
    result_final = result.copy()

    result_final['date'] = (
    np.where(result_final['0_x'].notnull(), result_final['0_x'],
        np.where((result_final['0_x'].isnull()) & (result_final['0_y'].notnull()), result_final['0_y'],
            np.where((result_final['0_x'].isnull()) & (result_final['0_y'].isnull()) & (result_final[0].notnull()), result_final[0],
                     result_final['0_x']
                    )))
    )

    # sort by final date
    result_final = result_final.sort_values(by = ['date'], ascending = True)

    # transform to series
    assignment1_result = result_final['level_0']
    
    return assignment1_result